In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. 라이브러리 및 데이터 세트 로딩

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

# 2. 필요한 데이터 불러오기

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')

In [ ]:
# 데이터 확인하기
print('Size of application_train data', app_train.shape)
print('Size of bureau data', bureau.shape)

# 3. 결측치 확인하기

In [ ]:
# Null 값 확인 가능한 사용자 함수 (데이터에서 비율)

def nulldata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    f,ax =plt.subplots(figsize=(15,10))
    plt.xticks(rotation='90')
    fig=sns.barplot(ms.index, ms["Percent"],color="green",alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of null values', fontsize=15)
    plt.title('Percent null data by feature', fontsize=15)
    return ms

In [ ]:
# missing data of app_train
nulldata(app_train)

# 4. EDA



In [ ]:
# Numerical Features(숫자형 피처) 과 Categorical Features(범주/object 피처) 구분 함수

def type_features(data):
    categorical_features = data.select_dtypes(include = ["object"]).columns
    numerical_features = data.select_dtypes(exclude = ["object"]).columns
    print( "categorical_features :",categorical_features)
    print('-----'*40)
    print("numerical_features:",numerical_features)

In [ ]:
# 시각화 함수
def plot_re(df,t1='',t2=''):
    f,ax=plt.subplots(1,2,figsize=(12,8))
    df[[t1,t2]].groupby([t1]).count().plot.bar(ax=ax[0],color='Blue')
    ax[0].set_title('count of customer on '+t1)
    sns.countplot(t1,hue=t2,data=df,ax=ax[1],palette="spring")
    ax[1].set_title(t1+': Target 0 vs Target 1')
    # Rotate x-labels
    plt.xticks(rotation=-90)
    a=plt.show()
    return a

In [ ]:
# columns에 따른 target 개수 함수
def show_count_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.countplot(df[cond_0][column], ax=ax[0][0]) # 대출 상환 o
        sns.countplot(df[cond_1][column], ax=ax[0][1]) # 대출 상환 x

## 1) application_train

In [ ]:
# 데이터 유형이 object인 컬럼을 target에 따라 count
object_column = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_column

In [ ]:
# object_column에 따른 target 개수 함수
show_count_by_target(app_train, object_column)

In [ ]:
# 자식 수(CNT_CHILDREN) 시각화
f,ax=plt.subplots(1,2,figsize=(12,6))
app_train.CNT_CHILDREN.value_counts().plot.pie(autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Distribution of CNT_CHILDREN')
ax[0].set_ylabel('')
sns.countplot('CNT_CHILDREN',data=app_train,ax=ax[1])
ax[1].set_title('CNT_CHILDREN')
plt.show()

# ===> 자식이 0명인 고객이 가장 많다.

In [ ]:
# 가족 수(CNT_FAM_MEMBERS) 시각화
f,ax=plt.subplots(1,2,figsize=(12,6))
app_train.CNT_FAM_MEMBERS.value_counts().plot.pie(autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Distribution of CNT_FAM_MEMBERS')
ax[0].set_ylabel('')
sns.countplot('CNT_FAM_MEMBERS',data=app_train,ax=ax[1])
ax[1].set_title('CNT_FAM_MEMBERS')
plt.show()

# ===> 대부분 2인 가구이고, 1인 가구, 3인 가구, 4인 가구 등이 있다.

In [ ]:
f, ax = plt.subplots(2,2,figsize=(13, 10))


# AMT_INCOME_TOTAL: 소득
sns.distplot(app_train.AMT_INCOME_TOTAL.dropna(), kde=True, color="g", 
             ax=ax[0,0]).set_title('AMT_INCOME_TOTAL Distribution')
# ===> 대부분의 고객은 소득이 없다.


# AMT_CREDIT: 대출 금액
sns.distplot(app_train.AMT_CREDIT.dropna(), kde=True, color="b",
             ax=ax[0,1]).set_title('AMT_CREDIT Distribution')


# DAYS_EMPLOYED: 대출 신청전 현 직업 유지 기간
sns.distplot(app_train.DAYS_EMPLOYED.dropna(), kde=True, 
             color="g", ax=ax[1,0]).set_title('DAYS_EMPLOYED Distribution')
# ===> 직업이 없는 고객이 많다.


# DAYS_BIRTH: 나이
sns.distplot(app_train.DAYS_BIRTH.dropna(), kde=True, color="b", 
             ax=ax[1,1]).set_title('DAYS_BIRTH Distribution')
# ===> 30-40대에 가장 많이 밀집되어 있다.



In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
# 젋을수록 대출 상환을 못함
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

### 나이에 따른 소득, 대출금액, 월 대출 지급액, 소비자 대출 상품액의 상관관계 알아보기

In [ ]:
# DAYS_BIRTH(나이), AMT_INCOME_TOTAL(소득), AMT_CREDIT(대출 금액), AMT_ANNUITY(월 대출 지급액),AMT_GOODS_PRICE(소비자 대출 상품액)

age_cols = app_train[['TARGET','DAYS_BIRTH', 'AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE']]
age_cols_corrs = age_cols.corr()
age_cols_corrs

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(age_cols_corrs, cmap='GnBu', annot=True)
plt.title('Correlation Heatmap by AGE')

### 성별에 따른 소득, 대출금액, 월 대출 지급액, 소비자 대출 상품액 상관관계

In [ ]:
# CODE_GENDER(성별), AMT_INCOME_TOTAL(소득), AMT_CREDIT(대출 금액), AMT_ANNUITY(월 대출 지급액), AMT_GOODS_PRICE(소비자 대출 상품액)

gender_cols = app_train[['TARGET','CODE_GENDER','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE']]
gender_cols_corrs = gender_cols.corr()
gender_cols_corrs

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(gender_cols_corrs, cmap='GnBu', annot=True)
plt.title('Correlation Heatmap by gender')

## 2) bereau

In [ ]:
# 데이터 확인하기
bureau.shape

In [ ]:
# TARGET 값을 가져오기 위해 bureau를 app_train과 조인
app_bureau = bureau.merge(app_train[['SK_ID_CURR', 'TARGET']], left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
app_bureau.shape

In [ ]:
# Target 비율
f,ax=plt.subplots(1,2,figsize=(12,6))
app_train.TARGET.value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Distribution of Target')
ax[0].set_ylabel('')
sns.countplot('TARGET',data=app_train,ax=ax[1])
ax[1].set_title('Target count')
plt.show()

In [ ]:
# 범주형과 숫자형으로 구분하기
type_features(app_bureau)

#### 범주형 피처 시각화

In [ ]:
# CREDIT_ACTIVE 대출 상태
plot_re(app_bureau,'CREDIT_ACTIVE','TARGET')

In [ ]:
# CREDIT_CURRENCY 대출 금액 화폐유형
plot_re(app_bureau,'CREDIT_CURRENCY','TARGET')

In [ ]:
# Credit type 카드 유형
plot_re(app_bureau,'CREDIT_TYPE','TARGET')

#### 숫자형 피처 시각화하기

In [ ]:
f, ax = plt.subplots(2,3,figsize=(13, 10))


# DAYS_CREDIT: 현재 대출 신청 일 기준 과거 대출 신청 지난 기간 분포 확인
sns.distplot(app_bureau.DAYS_CREDIT.dropna(), kde=True, color="g", 
             ax=ax[0,0]).set_title('DAYS CREDIT Distribution')

# CREDIT_DAY_OVERDUE: 대출 신청 시 CB 크레딧 연체 일수
sns.distplot(app_bureau.CREDIT_DAY_OVERDUE.dropna(), kde=True, color="b",
             ax=ax[0,1]).set_title(' CREDIT DAY OVERDUE Distribution')


# DAYS_CREDIT_UPDATE: 대출 신청전 마지막 정보 받은 기간
sns.distplot(bureau.DAYS_CREDIT_UPDATE.dropna(), kde=True, 
             color="r", ax=ax[0,2]).set_title('DAYS CREDIT UPDATE Distribution')


# DAYS_CREDIT_ENDDATE: 채무 완료까지 남아있는 채무일
sns.distplot(app_bureau.DAYS_CREDIT_ENDDATE.dropna(), kde=True, color="g", 
             ax=ax[1,0]).set_title('DAYS_CREDIT_ENDDATE Distribution')

# DAYS_ENDDATE_FACT: 채무 완료까지 실제 걸린 일자
sns.distplot(app_bureau.DAYS_ENDDATE_FACT.dropna(), kde=True, color="b",
             ax=ax[1,1]).set_title('DAYS_ENDDATE_FACT Distribution')


# CNT_CREDIT_PROLONG: 신용 연장 횟수
sns.distplot(bureau.CNT_CREDIT_PROLONG.dropna(), kde=True, 
             color="r", ax=ax[1,2]).set_title('CNT_CREDIT_PROLONG Distribution')


# 5. Feature Engineering
- 한 사람 당 대출 횟수는?
- 고객 당 과거 대출 유형 수
- 대출 상태의 현재 대출 비율
- 채무 완료까지 남아 있는 일수 - 채무 완료일까지 실제 걸린 횟수 ==> 차이가 클수록 좋을까?
- 현재 크레딧 금액 총액 / 현재 채무 금액 총액

In [ ]:
# 상관관계
corrmat = app_bureau.corr()
plt.figure(figsize=(12,8))
g = sns.heatmap(corrmat,annot=True,cmap="GnBu")

### 1) 한 사람당 대출 횟수

In [ ]:
# 한 사람 당 대출 횟수
bureau_fe1= bureau
PAST_LOANS_PER_CUS= bureau_fe1[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
bureau_fe1 = bureau_fe1.merge(PAST_LOANS_PER_CUS, on = ['SK_ID_CURR'], how = 'left')
print(bureau.shape)
print(bureau_fe1.shape)

### 2) 대출 횟수에 따른 신용 연장 횟수

In [ ]:
CNT_BUREAU_PROLONG = bureau_fe1[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']].groupby(by = ['SK_ID_CURR'])['CNT_CREDIT_PROLONG'].nunique().reset_index().rename(index=str, columns={'CNT_CREDIT_PROLONG': 'CNT_BUREAU_PROLONG'})
bureau_fe1 = bureau_fe1.merge(CNT_BUREAU_PROLONG, on = ['SK_ID_CURR'], how = 'left')
print(bureau_fe1.shape)
bureau_fe1.head()

### 3) 대출 상태의 현재 대출 비율

In [ ]:
# BUREAU 데이터의 active 대출 비율

bureau_fe1['CREDIT_ACTIVE_CLOSED'] = bureau_fe1['CREDIT_ACTIVE']

def f(x):
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

bureau_fe1['CREDIT_ACTIVE_CLOSED'] = bureau_fe1.apply(lambda x: f(x.CREDIT_ACTIVE), axis = 1)

grp = bureau_fe1.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_CLOSED'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_CLOSED':'ACTIVE_LOANS_PERCENTAGE'})
bureau_fe1= bureau_fe1.merge(grp, on = ['SK_ID_CURR'], how = 'left')

# 중복된 컬럼 삭제하기
del bureau_fe1['CREDIT_ACTIVE_CLOSED']
print(bureau_fe1.shape)

### 4) 채무 완료까지 남아있는 일수 - 채무 완료일까지 실제 걸린 횟수

In [ ]:
# DAYS_CREDIT_ENDDATE null 확인
bureau_fe1['DAYS_CREDIT_ENDDATE'].isnull().sum()

In [ ]:
# DAYS_CREDIT_ENDDATE 결측치에 평균값 대입하기
bureau_fe1['DAYS_CREDIT_ENDDATE'] = bureau_fe1['DAYS_CREDIT_ENDDATE'].fillna(bureau_fe1['DAYS_CREDIT_ENDDATE'].mean())
bureau_fe1['DAYS_CREDIT_ENDDATE'].isnull().sum()

In [ ]:
# DAYS_ENDDATE_FACT null 확인
bureau_fe1['DAYS_ENDDATE_FACT'].isnull().sum()

In [ ]:
# DAYS_ENDDATE_FACT 결측치에 평균값 대입하기
bureau_fe1['DAYS_ENDDATE_FACT'] = bureau_fe1['DAYS_ENDDATE_FACT'].fillna(bureau_fe1['DAYS_ENDDATE_FACT'].mean())
bureau_fe1['DAYS_ENDDATE_FACT'].isnull().sum()

In [ ]:
bureau_fe1['CREDIT_D_DAY'] = bureau_fe1['DAYS_CREDIT_ENDDATE']-bureau_fe1['DAYS_ENDDATE_FACT']
bureau_fe1['CREDIT_D_DAY']

### 5) 현재 크레딧 금액 총액 / 현재 채무 금액 총액

In [ ]:
# AMT_CREDIT_SUM null 확인
bureau_fe1['AMT_CREDIT_SUM'].isnull().sum()

In [ ]:
# AMT_CREDIT_SUM 결측치에 평균값 대입하기
bureau_fe1['AMT_CREDIT_SUM'] = bureau_fe1['AMT_CREDIT_SUM'].fillna(bureau_fe1['AMT_CREDIT_SUM'].mean())
bureau_fe1['AMT_CREDIT_SUM'].isnull().sum()

In [ ]:
# AMT_CREDIT_SUM_DEBT null 확인
bureau_fe1['AMT_CREDIT_SUM_DEBT'].isnull().sum()

In [ ]:
# AMT_CREDIT_SUM_DEBT 결측치에 평균값 대입하기
bureau_fe1['AMT_CREDIT_SUM_DEBT'] = bureau_fe1['AMT_CREDIT_SUM_DEBT'].fillna(bureau_fe1['AMT_CREDIT_SUM_DEBT'].mean())
bureau_fe1['AMT_CREDIT_SUM_DEBT'].isnull().sum()

In [ ]:
bureau_fe1['AMT_CREDIT_DEBT'] = bureau_fe1['AMT_CREDIT_SUM']/bureau_fe1['AMT_CREDIT_SUM_DEBT']
bureau_fe1['AMT_CREDIT_DEBT']

In [ ]:
bureau_fe1 = bureau_fe1.merge(app_bureau, on='SK_ID_CURR', how='left')
print(bureau_fe1.shape)